In [1]:
# Header를 통해서 TCP/IP를 확인 가능하다
# 하지만 아무 데이터나 수집하면 안되므로 확인해야 할 것이 존재함
# 1. robots.txt
# 2. traffic => 너무 많은 딜레이는 서버에 과부하를 주어 DDoS와 같은 효과가 나타날 수 있기 떄문이다
# 3. 이용약관
# 4. 저작권 걸린 DB에 존재하는 애들
# 5. 개인정보
# 개인이 공적, 연구용으로 배포하지 않고 공유하지 않으면 괜찮다

# Header
# Status, Reason 반드시 확인
# Content-type 또한 encode를 위해서 확인
# User-agent 또한 없어서 접근이 안되는 경우가 있기에 꼭 확인해야함

# HTTP<urllib><Requests> - Request가 high-level
# - URL parsing: Byte를 주고 받는데 한글 또는 JSON을 어케 넘길 것인가

# Requests의 장점
# 1. response 객체 - 헤더도 있고 자동으로 변환해주고 encoding도 알아서 변환해줌
#   request('method 방식', url, params={딕셔너리 형태}, data={딕셔너리 형태})
#   [request를 사용해서 response 객체를 생성
#    request를 통해서 status와 reason을 확인 가능함
#                    2xx,4xx,..  OK,..
#    resp.status_code, resp.reason]
# 2. request 객체 - 한 번에 필요한 파라미터들을 집어넣을 수 있어서 편하다
#   
# HTTP Error - 에러가 나도 헤더에 값을 담고 있음, raise_for_status()를 try except 구문으로 확인 가능

In [2]:
from requests import request

url = 'https://httpbin.org/post'
resp = request('POST', url, params={'key':'value'}, data={'이름':'값'})

In [3]:
resp.status_code, resp.reason

(200, 'OK')

In [4]:
resp.headers['content-type']

'application/json'

In [5]:
import json
# json은 왜? application의 json type이 존재하기 떄문이다

In [6]:
# json.load (fp), loads (str)
# 불러올때

# json.dump(fp), dumps(str)
# 저장할때

In [7]:
json.loads(resp.text)['args'].keys(),\
json.loads(resp.text)['args'].values()   # resp를 text로 변환후 넘겨줌
# 우리가 전달한 params 값이 들어가는 곳

(dict_keys(['key']), dict_values(['value']))

In [8]:
json.loads(resp.text)['form']
# 우리가 전달한 data값이 들어가는 곳

{'이름': '값'}

In [9]:
resp.request.body

'%EC%9D%B4%EB%A6%84=%EA%B0%92'

In [10]:
from requests.compat import urlparse
urlparse(resp.request.url).query, resp.request.body
#  query string                 form-data
# params를 어디에 넣은건지 볼 수 있는 것
# params를 body에 넣는 방법이 post 혹은 put일 뿐임
# 넘어가는 형태는 같다
# key=value를 이름1 = 값1 로 바꿔보면 앎

('key=value', '%EC%9D%B4%EB%A6%84=%EA%B0%92')

In [11]:
# form-data는 form에 저장되어 있기 떄문이다
# form은 html에 저장되는 tag를 의미함
# form안에 <input> 값에 name과 value attribute를 의미함
# POST, pUT은 body를 사용하는 것이고
# GET은 body를 사용 못하므로 주소로 넘기는 것이다

In [15]:
ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
url = 'https://www.google.com/search'
params = {
    'q': '수지', 
    'sourceid':'chrome',
    'ie':'UTF-8'
}
resp = request('GET', url, params=params, headers={'user-agent': ua})
resp.status_code, resp.reason

(200, 'OK')

In [16]:
resp.headers, resp.request.headers

({'Content-Type': 'text/html; charset=UTF-8', 'Date': 'Wed, 13 Mar 2024 01:22:42 GMT', 'Expires': '-1', 'Cache-Control': 'private, max-age=0', 'Strict-Transport-Security': 'max-age=31536000', 'Content-Security-Policy': "object-src 'none';base-uri 'self';script-src 'nonce-Ia3lyycp9prAPlKMsLMNxg' 'strict-dynamic' 'report-sample' 'unsafe-eval' 'unsafe-inline' https: http:;report-uri https://csp.withgoogle.com/csp/gws/cdt1", 'Cross-Origin-Opener-Policy': 'same-origin-allow-popups; report-to="gws"', 'Report-To': '{"group":"gws","max_age":2592000,"endpoints":[{"url":"https://csp.withgoogle.com/csp/report-to/gws/cdt1"}]}', 'Accept-CH': 'Sec-CH-UA-Platform, Sec-CH-UA-Platform-Version, Sec-CH-UA-Full-Version, Sec-CH-UA-Arch, Sec-CH-UA-Model, Sec-CH-UA-Bitness, Sec-CH-UA-Full-Version-List, Sec-CH-UA-WoW64', 'Permissions-Policy': 'unload=()', 'Origin-Trial': 'Ap+qNlnLzJDKSmEHjzM5ilaa908GuehlLqGb6ezME5lkhelj20qVzfv06zPmQ3LodoeujZuphAolrnhnPA8w4AIAAABfeyJvcmlnaW4iOiJodHRwczovL3d3dy5nb29nbGUuY29tOjQ

In [19]:
# 해석 가능
# resp.text # 비록 한글은 깨져있다
# 한글 깨지는걸 방지하려면 decode
from requests.compat import unquote # 한글이 깨졌을 때
from html import unescape   # html이 깨졌을 때
import re

re.findall(r'<h3 class="LC20lb MBeuO DKV0Md">(.+?)</h3>', resp.text)
# <h3 class="LC20lb MBeuO DKV0Md">수지(1994)</h3>

# resp.text
# text에서 일치하는게 8개 뜨는 이유는?
# 이게 default 값이기 때문이다

['수지(1994)',
 '수지 (1994년) - 위키백과, 우리 모두의 백과사전',
 '수지',
 '수지 (Suzy)- 아티스트채널 - 멜론',
 '수지 - 위키백과, 우리 모두의 백과사전',
 "다시 아이돌 된 수지 “상처 많은 '이두나' 안아주고 싶어”",
 '"제가 죽어야겠네요"... 가수 수지, 연예인 동료들에 눈물 ...',
 '수지구청']

In [22]:
# 개발자 도구에서 보이는 elements들과 우리가 받음 resp.text는 다르다

# link 정보가 필요하다

re.findall(r'<a jsname="UWckNb" href="(.+?)"[^>]+?><br><h3 class="LC20lb MBeuO DKV0Md">(.+?)</h3>', resp.text)
# 수지 를 가지고 있는 link들을 가져오는 것들만

[('https://namu.wiki/w/%EC%88%98%EC%A7%80(1994)', '수지(1994)'),
 ('https://ko.wikipedia.org/wiki/%EC%88%98%EC%A7%80_(1994%EB%85%84)',
  '수지 (1994년) - 위키백과, 우리 모두의 백과사전'),
 ('https://namu.wiki/w/%EC%88%98%EC%A7%80', '수지'),
 ('https://m2.melon.com/artist/song.htm?artistId=514741',
  '수지 (Suzy)- 아티스트채널 - 멜론'),
 ('https://ko.wikipedia.org/wiki/%EC%88%98%EC%A7%80',
  '수지 - 위키백과, 우리 모두의 백과사전'),
 ('https://www.hani.co.kr/arti/culture/culture_general/1112601.html',
  "다시 아이돌 된 수지 “상처 많은 '이두나' 안아주고 싶어”"),
 ('https://www.autotribune.co.kr/news/articleView.html?idxno=12365',
  '"제가 죽어야겠네요"... 가수 수지, 연예인 동료들에 눈물 ...'),
 ('https://www.sujigu.go.kr/', '수지구청')]

In [23]:
url = 'https://search.naver.com/search.naver'
params = {
    'where': 'nexearch',
    'sm':'top_hty',
    'fbm': '0',
    'ie':'utf8',
    'query':'%EC%88%98%EC%A7%80'
}
resp = request('GET', url, params = params)
resp.status_code, resp.reason

(200, 'OK')

In [25]:
resp.headers, resp.request.headers, resp.request.url, resp.request.body
# 이때는 ua를 넣지 않았으므로 python이 ua로써 뜨게 된다
# 또한, 확인해야 할 것은 headers에서 UTF-8을 확인해야한다
# 마지막으로 나오는 주소값에 params가 잘 들어갔는지 또한 확인해야함

({'Date': 'Wed, 13 Mar 2024 01:38:43 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': 'page_uid=iQZvowqptbNssPwULvsssssstGG-497445; path=/; domain=.naver.com, _naver_usersession_=DZMqvocFcpBbWbTI+2gdyg==; path=/; expires=Wed, 13-Mar-24 01:43:43 GMT; domain=.naver.com', 'X-Frame-Options': 'SAMEORIGIN', 'X-XSS-Protection': '1; report=/p/er/post/xss', 'Cache-Control': 'no-cache, no-store, must-revalidate, max-age=0', 'Pragma': 'no-cache', 'Referrer-Policy': 'unsafe-url', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Server': 'nxg', 'Accept-CH': 'Sec-CH-UA, Sec-CH-UA-Arch, Sec-CH-UA-Bitness, Sec-CH-UA-Full-Version-List, Sec-CH-UA-Mobile, Sec-CH-UA-Model, Sec-CH-UA-Platform, Sec-CH-UA-Platform-Version, Sec-CH-UA-WoW64'},
 {'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive'},
 'https://search.naver.com/search.naver?where=nexearch&sm=top

In [26]:
resp.text

# 해야할 일, class name으로 몇개 있는지 확인하고 잘 나오는지 다시 확인해야함

'<!doctype html> <html lang="ko"><head> <meta charset="utf-8"> <meta name="referrer" content="always">  <meta name="format-detection" content="telephone=no,address=no,email=no"> <meta property="og:title" content="%EC%88%98%EC%A7%80 : 네이버 통합검색"/> <meta property="og:image" content="https://ssl.pstatic.net/sstatic/search/common/og_v3.png"> <meta property="og:description" content="\'%EC%88%98%EC%A7%80\'의 네이버 통합검색 결과입니다."> <meta name="description" lang="ko" content="\'%EC%88%98%EC%A7%80\'의 네이버 통합검색 결과입니다."> <title>%EC%88%98%EC%A7%80 : 네이버 통합검색</title> <link rel="shortcut icon" href="https://ssl.pstatic.net/sstatic/search/favicon/favicon_191118_pc.ico">  <link rel="search" type="application/opensearchdescription+xml" href="https://ssl.pstatic.net/sstatic/search/opensearch-description.https.xml" title="Naver" /><script> if (top.frames.length!=0 || window!=top) window.open(location, "_top"); </script><link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/sstatic/search/pc/css/sea

In [29]:
re.findall(r'<a href="([^"]+?)" class="news_tit"[^>]+?>(.+?)</a>',  resp.text)

[('https://www.allurekorea.com/2024/03/13/%ec%98%a4%eb%a5%b4%ea%b0%80%ec%a6%98%ec%9c%bc%eb%a1%9c-%ec%99%84%ec%84%b1%eb%90%98%eb%8a%94-%ec%84%b9%ec%8a%88%ec%96%bc-%ec%9b%b0%eb%8b%88%ec%8a%a4/?utm_source=naver&utm_medium=partnership',
  '오르가즘으로 완성되는 섹슈얼 웰니스'),
 ('https://www.allurekorea.com/2024/03/12/%ec%85%80%eb%9f%bd%eb%93%a4-sns-%ec%86%8d-%ea%b7%b8-%ec%b9%b4%ed%8e%98%ea%b0%80-%ea%b6%81%ea%b8%88%ed%95%b4/?utm_source=naver&utm_medium=partnership',
  '셀럽들 SNS 속 그 카페가 궁금해?'),
 ('https://www.allurekorea.com/2024/03/12/%ec%b9%98%eb%a3%8c%eb%b3%b4%eb%8b%a4-%ec%98%88%eb%b0%a9-%ea%b8%b0%eb%8a%a5-%ec%9d%98%ed%95%99/?utm_source=naver&utm_medium=partnership',
  '치료보다 예방, 기능 의학'),
 ('https://www.allurekorea.com/2024/03/11/%ec%a0%a4%eb%9d%bc%eb%b9%84%ea%b0%80-%ec%98%b7-%ec%9e%85%eb%8a%94-%eb%b2%95/?utm_source=naver&utm_medium=partnership',
  '젤라비가 옷 입는 법')]

In [35]:
re.findall(r'<a target="_blank" href="([^"]+?)" class="link_tit"[^>]+?>(.+?)</a>', resp.text)
# link_tit로 뽑아봄

[('https://hsmoa.com/search?query=%25EC%259D%25B8%25EB%258D%2595%25EC%2585%2598%25EA%25B0%2580%25EB%25A7%2588%25EC%2586%25A5',
  '<mark>%EC%</mark>9D%B8%EB%8D%95<mark>%EC%</mark>85%<mark>98%</mark>EA%B0%<mark>80</mark>%EB%<mark>A7%</mark><mark>88%</mark><mark>EC%</mark>86%A5 검색결과 | 홈쇼핑모아'),
 ('https://www.kfi.or.kr/pdf/html/web/viewer.html?file=%2Fportal%2Fcmmn%2Ffile%2FtechFileDown.do%3Fseq%3D317%26seqHistory%3D1415',
  '<mark>%EC%</mark>9C%84%ED%97%<mark>98%</mark>EB%AC%BC<mark>%EC%</mark>9D%<mark>98 %</mark>ED%8C%90<mark>%EC%</mark>A0%95 %EB%B0%8F <mark>%EC%</mark><mark>A7%80</mark><mark>%EC%</mark><mark>A</mark> - 한국소방산업기술원')]

In [36]:
url = 'https://search.daum.net/search'
params = {
    'w':'tot',
    'DA':'SBC',
    'q':'수지',
}
resp = request('GET', url, params=params)
resp.status_code, resp.reason

(200, 'OK')

In [37]:
resp.text

'<!doctype html>\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ko">\n<head profile="http://a9.com/-/spec/opensearch/1.1/">\n<meta http-equiv="content-Type" content="text/html;charset=utf-8" />\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\n<meta name="autocomplete" content="off" />\n<meta name="referrer" content="always">\n<meta name="format-detection" content="telephone=no" />\n<meta property="og:title" content="수지 &ndash; Daum 검색" />\n<meta property="og:url" content="https://search.daum.net/search?w=tot&amp;q=%EC%88%98%EC%A7%80" />\n<meta property="og:description" content="Daum 검색에서 수지에 대한 최신정보를 찾아보세요." />\n<meta property="og:image" content="https://search1.daumcdn.net/search/statics/common/img/og_search.png" />\n<meta property="og:site_name" content="다음검색" />\n<title>수지 &ndash; Daum 검색</title>\n<link rel="search" type="application/opensearchdescription+xml" href="//search.daum.net/OpenSearch.xml" title="Daum">\n<link rel="stylesheet" type="text/css" href=\'//search1.d

In [44]:
re.findall(r'<c-menu-share .+? data-title="(.+?)" .+? data-href="(.+?)"[^>]+?>(.+?)</c-title>', resp.text)

[('수지(1994) - 나무위키',
  'https://namu.wiki/w/%EC%88%98%EC%A7%80(1994)',
  '<b>수지</b>(1994) - 나무위키'),
 ('수지', 'http://100.daum.net/encyclopedia/view/156XX58614229', '<b>수지</b>'),
 ('헉 수지도 겟어기타 자주 듣는대..... 🎸',
  'https://table.cafe.daum.net/p/1311136016/238037326848740864',
  '헉 <b>수지</b>도 겟어기타 자주 듣는대..... 🎸'),
 ('[뷰티는 내운명] 청바지의 계절 봄, &amp;#39;국민 첫사랑&amp;#39; 수지 헤어 스타일링 - 콘텐츠뷰',
  'https://v.daum.net/v/6ut8sPf5q7',
  '[뷰티는 내운명] 청바지의 계절 봄, &#39;국민 첫사랑&#39; <b>수지</b> 헤어 스타일링 - 콘텐츠뷰'),
 ('수지 친언니가 찍어준 수지 사진들',
  'https://cafe.daum.net/SoulDresser/FLTB/789653?q=%EC%88%98%EC%A7%80&re=1',
  '<b>수지</b> 친언니가 찍어준 <b>수지</b> 사진들'),
 ('수지 시계화보 같은 포즈 2018 vs 2023',
  'https://table.cafe.daum.net/p/1000055110/237433615722087168',
  '<b>수지</b> 시계화보 같은 포즈 2018 vs 2023'),
 ('한국 무역수지 세계 200등? ㅠㅠ',
  'https://table.cafe.daum.net/p/1019101842/241378098830594304',
  '한국 무역<b>수지</b> 세계 200등? ㅠㅠ'),
 ('김은숙X수지X김우빈 &amp;#39;다 이루어질지니&amp;#39;',
  'https://table.cafe.daum.net/p/1105521742/192706843079813632',
  '김은숙X

In [45]:
url = 'https://search.naver.com/search.naver'
params = {
    "where":"nexearch",
    "sm":"top_hty",
    "fbm":"0",
    "ie":"utf8",
    "query":"수지"
}
resp = request('GET', url, params=params)
resp.status_code, resp.reason

(200, 'OK')

In [49]:
re.findall(r'<img data-image-viewer-trigger .+? src="(.+?)"', resp.text)

['https://search.pstatic.net/common/?src=http%3A%2F%2Fimgnews.naver.net%2Fimage%2F5129%2F2023%2F07%2F28%2F1690461044_19211135-6_20230728070403219.jpg&type=ff332_332',
 'https://search.pstatic.net/common/?src=http%3A%2F%2Fimgnews.naver.net%2Fimage%2F5902%2F2023%2F03%2F16%2F0000000033_001_20230316093803292.jpg&type=ff332_332',
 'https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyNDAyMDFfODAg%2FMDAxNzA2Nzk3NzU1NTc0.HNkgYUJ2UMgTqX6RDetG6_vepjkXj4Xw3zbhANSM9P0g.A3hUHFhnX7Tv4b2E-OfD1iuCdksH_7r1S1b9W4Qi-CAg.JPEG.dlaalwlsfjq%2F1706797748458.jpg&type=ff332_332',
 'https://search.pstatic.net/common/?src=http%3A%2F%2Fimgnews.naver.net%2Fimage%2F011%2F2023%2F10%2F18%2F0004250393_001_20231018112607650.jpg&type=ff332_332',
 'https://search.pstatic.net/common/?src=http%3A%2F%2Fimgnews.naver.net%2Fimage%2F082%2F2023%2F07%2F27%2F0001224312_001_20230727152901140.jpg&type=ff332_332',
 'https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyMzA5MTRfMTQg%2F

In [ ]:
# <tagname\s*.+?=.+?*> ==> tag의 속성도 가져올 수 있다 (key, value)

# .+? 어떤 글자들이 나오고
# [^>] 꺽쇠가 나오기 전까지
# (.+?) 우리가 구하려는 값의 장소

In [53]:
data = '''
<a href="/" asdf=asdfasdf>
'''
re.findall(r'<\w+(\s*.+?=.+?)*>',data)

# 한번 해봐~

['/" asdf=asdfasdf']

In [ ]:
# 검색결과(구글, 네이버, 다음)에서 링크와 제목 가져왔음
# 네이버는 이미지 링크 가져올 수 있었다
# 이 모든 것을 정규식으로

In [ ]:
# DOM
# Document Obejct Model
# Node의 객체 단위로 tree 형태로 표현하게 만들어줌
# 이때, 태그 단위로 Node가 된다

# BeautifulSoup
# parser 지원
# parse 진행 후 서로 다른 DOM을 만들어줌
# DOM의 tree 구조를 따르기 떄문에 관계에 따라서 탐색한다

In [55]:
from bs4 import BeautifulSoup

In [ ]:
html = '''
<html>
    <head></head>
    <body>
        <div>
            <ul>
            <li> 1st
            <li> 2nd
        </div>
    </ul>
    </body>
</html>
'''

In [57]:
dom1 = BeautifulSoup(data, 'html.parser')
dom2 = BeautifulSoup(data, 'lxml')
dom3 = BeautifulSoup(data, 'html5lib')
# 각각을 DOM으로 만드는 함수

In [59]:
dom2

# 지금 모든것들은 태그를 안닫았을때 그리고 제대로 닫았을 때 어떻게 진행되는지를 보는 것이다
# 각각은 html은 태그를 제대로 안닫아도 알아서 닫아주는 not well-formed지만
# xml은 아니다

<html><body><a asdf="asdfasdf" href="/">
</a></body></html>

In [65]:
html ='''
<html>
    <head>
    </head>
    <body>
        <div>
            <p>
                <a href = "/"> go to page </a>
                <a href = "/*"> go to page2 </a>
            </p>
            <p>
                <h2>Hi</h2>
            </p>
        </div>
    </body>
</html>
'''

In [66]:
dom1 = BeautifulSoup(data, 'html.parser')
dom2 = BeautifulSoup(data, 'lxml')
dom3 = BeautifulSoup(data, 'html5lib')
# 각각을 DOM으로 만드는 함수

In [68]:
dom3

<html><head></head><body><a asdf="asdfasdf" href="/">
</a></body></html>

In [69]:
dom = BeautifulSoup(html, 'html.parser')

In [73]:
type(dom), dom.html, dom.html.body
# 각각은 객체이다
# dom.html.body는 태그를 의미함

dom.html.body.div.p.a
# body - div - p - a를 찾아라 라는 뜻

<a> go to page </a>

In [75]:
dir(dom.a)

# 이 중에서 find 애들을 잘 봐야함
# html 태그에서 중요한건 has_attr

dom.a.attrs
# 이것은 a 태그가 가지고 있는 attr들을 dict로 보여준다

dom.a.attrs['href'], dom.a.get_text()
# 1. href 링크를 가지고 옴
# 2. 태그가 가지고 있는 text를 가지고 옴

# 문제점
# 링크가 두개면?

{}

In [77]:
# find는 자식들을 찾는 것
# find_parent 내 부모 찾는 것
# find_next_silbling, find_previous_sibling 같은 부모를 공유하는 자식들을 찾는 것
# find는 하나만
# find_all은 전체 찾는 것
dom.p.find()

<a> go to page </a>

In [78]:
dom.p.find_all(re.compile('h[1-6]'))
# 이런식으로 정규식을 쓸 수 있다

# 1 - 태그이름(텍스트, 정규식) 텍스트 또는 정규식
dom.p.find_all('a')
dom.p.find_all(re.compile('h[1-6]'))

# 2 - attr로 차즌ㄴ 것
dom.p.find_all(attrs={'href':'/'})  # 그냥 넣기
dom.p.find_all(attrs={'href': re.compile('htmls')}) # 정규식으로 넣어주기
# 당연히 href만 있는게 아니라 class 등등이 있다

# 3 - 자식/자손?
dom.p.find_all(recursive=False) # 다 표시가 됨 그래서...
[tag.name for tag in dom.p.find_all(recursive=False)]   # 이렇게 하면 태그만 볼 수 있다 (정확히는 태그의 이름)
# True로 바꾸면
# 나머지 자손들을 다 본다
# 즉 부모의 tag들만 보는게 아니라 부모 자식들의 태그들 모두 볼 수 있는 것이다

# 4 - 본문 내용을 찾기
dom.p.find_all(string='go to page')
dom.p.find_all(string = re.compile('3|4$')) # 정규식도 쓸 수 있다

# 정규식을 같이 쓰면 더 정교하고 자세하게 잘 찾을 수 있다

# 5 - 자손 중의 몇개까지 찾기
dom.p.find_all(limit=2) # limit=2는 자손 2개까지 찾겠다는 의미이다

# * - kwargs, attribute로 필터링

[<a> go to page </a>]

In [83]:
dom.p.find_parent(name='html')

# recursive가 없음
# limit 옵션은 존재함
dom.p.find_parents(limit=2)

[<div>
 <p>
 <a> go to page </a>
 </p></div>,
 <body>
 <div>
 <p>
 <a> go to page </a>
 </p></div>
 </body>]

In [ ]:
dom.a.find_next_sibling
dom.a.find_next_siblings
dom.a.find_previous_sibling
dom.a.find_previous_siblings

# div > p > a, a, div
#           *
dom.a.find_next_siblings()
# a, div 가져오는 것

# 여기까지...

# How to get Specific values?

## Roll-Back

Through header we can check TCP/IP

Regulation

1. robots.txt
2. traffic => too much traffic can harm server
3. Usage policy
4. DB data 
5. Personal Data

* But it is okay to use data in purpose of personal and research

#### Header

Check **Status, Reason**

Check **Content-type** for encoding

Check **User-agent** for access

#### Structure

HTTP > urllib > requests / requests is high-level

### Request

Advantage

1. response 객체 - 헤더 O / 자동으로 헤더 변환 / encoding 자동 변환
    request('method 방식', url, params={딕셔너리 형태}, data={딕셔너리 형태})
    [request를 사용해서 response 객체를 생성
     request를 통해서 status와 reason을 확인 가능함
                     2xx,4xx,..  OK,..
     resp.status_code, resp.reason]
2. request 객체 - 한 번에 파라미터를 넣을 수 있다
    하지만 보낼때 HTTP Error을 알 수 있다 => raise_for_status()로 확인 가능

## JSON

Content-type은 다양하게 존재한다 그 중에서 JSON으로 보내는 법을 알아보자

```
import json

json.load(fp) , loads(str)

json.dump(fp), dumps(str)
```

1. load() - 불러올때
    * fp - parameter - file
    * str - paramerter is string
2. dump() - 저장할때
    * 위와 같음

Json has key and value

```
json.loads(resp.text)['args'].keys()
json.loads(resp.text)['args'].values()
```

Key and values are **parameters** that we send

```
json.loads(resp.text)['form']
```

form is data that we send

##### Query String and Form-Data

```
from requests.compat import urlparse

urlparse(resp.request.url).query
# query string

resp.request.body
# form-data
```

Query string - the string after question mark ?

Form-data - character inside of the format

정리하자면 QS는 지정 타입이다

FD는 타입에 맞게 들어가는 value라고 생각하면 된다

자세히는 form-data는 form 태그 안에 저장되는 것을 의미함

그 중에서 input 값에 name과 value attribute를 의미함

#### RESTful Method

POST, PUT - Use body

GET - body를 사용 못하므로 주소로 넘김

## Request Library

request.text - 한글이 깨진다 따라서 decode를 진행해줘야 함

resp.text에서 Regular Expression으로 원하는 값만 뽑을 수 있다

만약 google에서 수지 관련된 link만 가지고 온다고 생각한다면...

```
url = 'https://www.google.com/search'
ua = 'Mozilla~'
params = {
    'q': '수지',
    'sourceid':'chrome',
    'ie':'UTF-8'
}

resp = request('GET', url, params=params, headers={'user-agent':ua})
# resp.status_code, resp.reason

re.findall(r'<h3 class="LC20lb MBeuO DKV0Md">(.+?)</h3>', resp.text)
# only text data

re.findall(r'<a jsname="UWckNb" href="(.+?)"[^>]+?><br><h3 class="LC20lb MBeuO DKV0Md">(.+?)</h3>', resp.text)
# link and text data
```

In params, we have to put proper params

* Regular Expression
    1. .+? 어떤 글자들이 나오고
    2. [^>] 꺽쇠가 나오기 전까지
    3. (.+?) 우리가 구하려는 값의 장소

## DOM

DOM = Document Object Model

태그를 Node로 단위로 변환시켜 tree 형태로 표현해줌

## BeautifulSoup

Why we use?

1. parser 지원
2. parser 진행 후 다른 DOM을 만들어줌
3. DOM tree 구조를 따라서 관계에 따라 탐색함

#### Functions Usage

```
from bs4 import BeautifulSoup

dom = BeautifulSoup(data, 'html.parser')
dom = BeautifulSoup(data, 'lxml')
dom = BeautifulSoup(data, 'html5lib')
# Change each of data into DOM by second parameter
```

각각의 dom은 객체가 되며 dom에서 html, body등 접근 가능하다

```
dom, dom.html, dom.html.body
dom.a   # a tag
dom.a.attrs # a가 가지고 있는 attr을 dict로 보여줌
dom.a.attrs['href'], dom.a.get_text()
# 하나의 href 링크 가지고옴 | 태그가 가지고 있는 text

dom.p.find()
# 자식들 찾는것
dom.p.find_parent()
# 부모 찾는것
dom.p.find_next_sibling()
# 하나의 같은 부모 공유하는 다음 자식
dom.p.find_next_siblings()
# 여러 같은 부모 공유하는 다음 자식
dom.p.find_previous_sibling(), dom.p.find_previous_siblings()
# 하나, 여러 같은 부모 공유하는 이전 자식

# siblings 구조
dom.a.find_next_siblings
# div > p > a, a, div       -->         div > p > a, a, div
#           *                                        *

# 옵션 추가
dom.p.find_all(re.compile('h[1-6]'))
# 이런식으로 정규식을 쓸 수 있다

# 1 - 태그이름(텍스트, 정규식) 텍스트 또는 정규식
dom.p.find_all('a')
dom.p.find_all(re.compile('h[1-6]'))

# 2 - attr로 차즌ㄴ 것
dom.p.find_all(attrs={'href':'/'})  # 그냥 넣기
dom.p.find_all(attrs={'href': re.compile('htmls')}) # 정규식으로 넣어주기
# 당연히 href만 있는게 아니라 class 등등이 있다

# 3 - 자식/자손?
dom.p.find_all(recursive=False) # 다 표시가 됨 그래서...
[tag.name for tag in dom.p.find_all(recursive=False)]   # 이렇게 하면 태그만 볼 수 있다 (정확히는 태그의 이름)
# reculsive = True로 바꾸면
# 나머지 자손들을 다 본다
# 즉 부모의 tag들만 보는게 아니라 부모 자식들의 태그들 모두 볼 수 있는 것이다

# 4 - 본문 내용을 찾기
dom.p.find_all(string='go to page')
dom.p.find_all(string = re.compile('3|4$')) # 정규식도 쓸 수 있다

# 정규식을 같이 쓰면 더 정교하고 자세하게 잘 찾을 수 있다

# 5 - 자손 중의 몇개까지 찾기
dom.p.find_all(limit=2) # limit=2는 자손 2개까지 찾겠다는 의미이다

# * - kwargs, attribute로 필터링

dom.p.find_parent(name='html')
# recursive 없음
# limit 옵션은 있음
dom.p.find_parents(limit=2)
```

